In [1]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
import pandas as pd
from scraper_tools import go_back,go_to_next_page,wait
import scraper_tools
from importlib import reload
reload(scraper_tools)

<module 'scraper_tools' from '/Users/cheetham/Dropbox/insight/ideas/jigsaw/scraper_tools.py'>

In [4]:
base_url = "https://www.amazon.com/s?i=toys-and-games&rh=n%3A165793011%2Cn%3A165795011%2Cn%3A166359011%2Cn%3A166363011%2Cp_n_feature_five_browse-bin%3A3136099011%2Cp_72%3A1248964011"
driver = webdriver.Chrome()
driver.get(base_url)
wait(amount=10) # So I can change it to the US store..

out_dir = '1000_piece/'

In [8]:
#

# Main loop through search pages:
for page_num in np.arange(0,0):
    # Load the page
    driver.get(base_url+"&page="+str(page_num))
    html = driver.page_source
    print(base_url+"&page="+str(page_num))
    raise Exception
    
    fname = out_dir+"results_page_{0:05}.html".format(page_num)
    with open(fname,'w') as myf:
        myf.write(html)
        
    wait(amount=2)
    print(' Done page {0}'.format(page_num))
    
print('Done!')

https://www.amazon.com/s?i=toys-and-games&rh=n%3A165793011%2Cn%3A165795011%2Cn%3A166359011%2Cn%3A166363011%2Cp_n_feature_five_browse-bin%3A3136099011%2Cp_72%3A1248964011&page=294


Exception: 

In [38]:
# Scrape the metadata from the saved pages
urls = []
for page_num in np.arange(1,294):
    fname = out_dir+"results_page_{0:05}.html".format(page_num)
    with open(fname,'r') as myf:
        html = myf.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    # Get the product urls
    results = soup.findAll('div',attrs={'class':'s-result-item'})

    # Loop through
    for item in results:
        item_class = item.get('class')
        # Skip the ads
        if 'AdHolder' in item_class:
            continue

        # Get the product page url and save it for later
        url = 'https://amazon.com'+item.find('a').get('href')
        
        # Strip off the trackers
        url = url.rsplit('/',1)[0]
        
        urls.append(url)
        
# Save them all
with open(out_dir+'product_urls.txt','w') as myf:
    for url in urls:
        myf.writelines(url+'\n')

In [57]:
reload(scraper_tools)
# Now scrape each one
with open(out_dir+'product_urls.txt','r') as myf:
    product_urls = myf.readlines()

# Set up the pandas array
# reviews_df = scraper_tools.get_database()

# reviews_df = reviews_df.drop([2360,2361,2362,2363,2364])

# test=0
# # Loop over product urls
# for url_ix,product_url in enumerate(product_urls):    
#     # Check if it's already filled:
#     try:
#         x = reviews_df['Name'][url_ix]
#         if isinstance(reviews_df['Name'][url_ix],str):
#             # Skip this one since it's already in the DB
#             continue
#         else:
#             pass
#     except:
#         pass
#     # Get the page metadata
#     driver.get(product_url)
    
#     item_info = scraper_tools.get_info(driver)
    
#     n_saved = reviews_df.shape[0]-1
    
#     if n_saved > url_ix:
#         for key in item_info.keys():
#             reviews_df[key][url_ix] = item_info[key]
#     else:
#         reviews_df = reviews_df.append(item_info,ignore_index=True)
    
#     #Save it
#     reviews_df.to_csv('review_db.csv')
#     print(' Metadata for # {0} (saved as {1})'.format(url_ix,reviews_df.shape[0]-1))
#     wait(amount=1)



In [48]:
# print(item_info)
# reviews_df = reviews_df.append(item_info,ignore_index=True)
# print(reviews_df.iloc[-1])
reviews_df.to_csv('review_db.csv')
# print(reviews_df.shape[0]-1)

In [52]:
print('Done!')
# print(x)
# fname = 'review_db.csv'
# reviews_df = pd.read_csv(fname)
# print(isinstance(reviews_df['Name'][url_ix],str))
# reviews_df.to_csv('review_db2.csv')

Done!


In [5]:
# Get reviews by replacing /dp/ with /product-reviews/ in the url!
product_urls = np.loadtxt(out_dir+'product_urls.txt',dtype=str)

# Set up the pandas array
reviews_df = scraper_tools.get_database()

for product_ix in np.arange(0,1):
    
    product_url = product_urls[product_ix]
    
    n_reviews = reviews_df['n_reviews'][product_ix]
    if n_reviews != n_reviews:
        n_reviews = '0 '
    n_reviews = n_reviews.split(' ')[0]
    n_reviews = int(n_reviews)
    
    n_pages = np.ceil(n_reviews/10).astype(int)
    
    all_ratings = []
    all_reviews = []
    
    print('  Product {0}: {1} pages'.format(product_ix,n_pages))
    
    for page_ix in range(n_pages):
        
        # Navigate to page
        review_url = product_url.replace('/dp/','/product-review/')
        review_url += '/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber={0}'.format(page_ix+1) # starts at 1
        driver.get(review_url)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Loop through reviews on the page
        reviews_on_page = soup.findAll('div',attrs={'data-hook':'review'})
        for r in reviews_on_page:
            # Get the star rating
            rating_text = r.find('i',attrs={'data-hook':"review-star-rating"}).getText()
            rating = float(rating_text[0:2])
            all_ratings.append(rating)

            # Get the review text
            review_text = r.find('span',attrs={'data-hook':"review-body"}).getText()
            all_reviews.append(review_text)
        
        wait(amount=1)
        
        product_df = pd.DataFrame({'ReviewRatings':all_ratings,'Reviews':all_reviews})
        product_df.to_csv(out_dir+'product_{0:05}.csv'.format(product_ix))
#         print(out_dir+'product_{0:05}.csv'.format(product_ix))


  Product 0: 92 pages


In [68]:
# print(len(all_ratings))
# print(reviews_df['Reviews'][0:5].dtype)
# print(n_reviews)
print(len(product_urls))
# print(n_pages)
# print(reviews_df['n_reviews'][product_ix])
# print(product_url)

6990
